source: https://kenbenoit.net/assets/courses/ME104/logmodels2.pdf

WBdata API https://wbdata.readthedocs.io/en/stable/

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
wb_regions = ['Arab World', 'Caribbean small states',
              'Central Europe and the Baltics', 'Early-demographic dividend',
              'East Asia & Pacific', 'East Asia & Pacific (excluding high income)',
              'East Asia & Pacific (IDA & IBRD countries)',
              'Euro area', 'Europe & Central Asia',
              'Europe & Central Asia (excluding high income)',
              'Europe & Central Asia (IDA & IBRD countries)',
              'European Union',
              'Fragile and conflict affected situations',
              'Heavily indebted poor countries (HIPC)',
              'High income',
              'IBRD only',
              'IDA & IBRD total',
              'IDA blend',
              'IDA only',
              'IDA total',
              'Late-demographic dividend',
              'Latin America & Caribbean',
              'Latin America & Caribbean (excluding high income)',
              'Latin America & the Caribbean (IDA & IBRD countries)',
              'Least developed countries',
              'Least developed countries: UN classification',
              'Low & middle income',
              'Low income',
              'Lower middle income',
              'Middle East & North Africa',
              'Middle East & North Africa (excluding high income)',
              'Middle East & North Africa (IDA & IBRD countries)',
              'Middle income',
              'North America',
              'Not classified',
              'OECD members',
              'Other small states',
              'Pacific island small states',
              'Post-demographic dividend',
              'Pre-demographic dividend',
              'Small states',
              'South Asia',
              'South Asia (IDA & IBRD)',
              'Sub-Saharan Africa',
              'Sub-Saharan Africa (excluding high income)',
              'Sub-Saharan Africa (IDA & IBRD countries)',
              'Upper middle income',
              'World']

In [4]:
import wbdata
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [ ]:
NY.GDP.PCAP.PP.CD
SP.URB.TOTL.IN.ZS 

In [ ]:
wbdata.search_indicators('urban')

In [9]:
wbdata.get_data('NY.GDP.PCAP.PP.CD')

[{'indicator': {'id': 'NY.GDP.PCAP.PP.CD',
   'value': 'GDP per capita, PPP (current international $)'},
  'country': {'id': '1A', 'value': 'Arab World'},
  'value': None,
  'decimal': '1',
  'date': '2019'},
 {'indicator': {'id': 'NY.GDP.PCAP.PP.CD',
   'value': 'GDP per capita, PPP (current international $)'},
  'country': {'id': '1A', 'value': 'Arab World'},
  'value': '17571.4131783648',
  'decimal': '1',
  'date': '2018'},
 {'indicator': {'id': 'NY.GDP.PCAP.PP.CD',
   'value': 'GDP per capita, PPP (current international $)'},
  'country': {'id': '1A', 'value': 'Arab World'},
  'value': '17101.1308307964',
  'decimal': '1',
  'date': '2017'},
 {'indicator': {'id': 'NY.GDP.PCAP.PP.CD',
   'value': 'GDP per capita, PPP (current international $)'},
  'country': {'id': '1A', 'value': 'Arab World'},
  'value': '16936.6128000594',
  'decimal': '1',
  'date': '2016'},
 {'indicator': {'id': 'NY.GDP.PCAP.PP.CD',
   'value': 'GDP per capita, PPP (current international $)'},
  'country': {'id

In [ ]:
raw_gdp_pcs = wbdata.get_data('NY.GDP.PCAP.PP.CD')
gdp_pcs = {}
for X in raw_gdp_pcs:
    date = X['date']
    country = X['country']['value']
    if date == '2018' and country not in wb_regions:
        try:
            gdp_pc = float(X['value'])
            gdp_pcs[country] = gdp_pc
        except:
            pass

In [ ]:
gdp_pcs

In [ ]:
raw_urban_pops = wbdata.get_data('SP.URB.TOTL.IN.ZS')
urban_pops = {}
for X in raw_urban_pops:
    date = X['date']
    country = X['country']['value']
    if date == '2018' and country not in wb_regions:
        try:
            urban_pop = float(X['value'])
            urban_pops[country] = urban_pop
        except:
            pass

In [ ]:
urban_pops

In [ ]:
countries = []
X = []
Y = []
log_X = []
for country in gdp_pcs.keys():
    if country in urban_pops.keys():
        x = gdp_pcs[country]
        y = urban_pops[country]
        x_log = np.log(x)
        X.append(x)
        Y.append(y)
        log_X.append(x_log)
        countries.append(country)        

In [ ]:
def scatterplot_X_Y(X, Y, X_title, Y_title, plot_regression=False):
    fig = plt.figure(figsize=(10, 10))
    plt.xlabel(X_title, fontsize=20)
    plt.ylabel(Y_title, fontsize=20)
    plt.plot(X, Y, 'o', markersize=5, color='red')
    min_x = min(X)
    min_y = min(Y)
    max_x = max(X)
    max_y = max(Y)
    if plot_regression:
        X = sm.add_constant(X)
        model = sm.OLS(Y, X).fit()
        beta_0 = model.params.tolist()[0]
        beta_1 = model.params.tolist()[1]
        R2 = model.rsquared
        plt.plot([0, max_x], [beta_0, beta_0 + (beta_1 * max_x)], color='black', linestyle='solid', linewidth=2)
        plt.title('R2: ' + str(R2), fontsize=20, fontweight='bold')
        plt.xlim(min_x, max_x)
        plt.ylim(min_y, max_y)
    plt.show()
    plt.close(plt.gcf())

In [ ]:
scatterplot_X_Y(X, Y, 'PIB Per Cápita', 'Población Urbana')

In [ ]:
scatterplot_X_Y(log_X, Y, 'log(PIB Per Cápita)', 'Población Urbana')

In [ ]:
scatterplot_X_Y(log_X, Y, 'log(PIB Per Cápita)', 'Población Urbana', True)

In [ ]:
log_X = sm.add_constant(log_X)
model = sm.OLS(Y,log_X).fit()
beta = model.params.tolist()[0]
R2 = model.rsquared
model.summary()